## 配置环境

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 14.5 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 776.8 MB 10.3 MB/s eta 0:00:01     |████▊                           | 114.0 MB 12.0 MB/s eta 0:00:56     |███████▎                        | 176.4 MB 10.5 MB/s eta 0:00:58     |█████████▉                      | 239.4 MB 12.0 MB/s eta 0:00:45     |█████████████████▌              | 425.8 MB 11.2 MB/s eta 0:00:32     |█████████████████▋              | 427.6 MB 11.2 MB/s eta 0:00:32     |███████████████████████▉        | 579.0 MB 10.2 MB/s eta 0:00:20     |██████████████████████████▊     | 648.7 MB 10.2 MB/s eta 0:00:13     |██████████████████████████▊     | 649.9 MB 10.2 MB/s eta 0:00:13     |███████████████████████████▊    | 672.3 MB 12.0 MB/s eta 0:00:09
     |████████████████████████████████| 12.8 MB 12.0 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninst

In [ ]:
***不要下openai的CLIP，baseline代码已经把CLIP打包好了，如果有问题要注意一下是不是相对路径的问题***

In [ ]:
#!pip install ftfy regex tqdm
#!pip install git+https://github.com/openai/CLIP.git

## 解压数据集和权重

In [3]:
!unzip -d ./data ./Multimodal_Retrieval.zip

Archive:  ./Multimodal_Retrieval.zip
   creating: ./data/Multimodal_Retrieval/
  inflating: ./data/__MACOSX/._Multimodal_Retrieval  
  inflating: ./data/Multimodal_Retrieval/MR_train_imgs.tsv  
  inflating: ./data/Multimodal_Retrieval/MR_test_queries.jsonl  
  inflating: ./data/__MACOSX/Multimodal_Retrieval/._MR_test_queries.jsonl  
  inflating: ./data/Multimodal_Retrieval/MR_train_queries.jsonl  
  inflating: ./data/Multimodal_Retrieval/MR_valid_queries.jsonl  
  inflating: ./data/Multimodal_Retrieval/README.txt  
  inflating: ./data/Multimodal_Retrieval/example_pred.jsonl  
  inflating: ./data/Multimodal_Retrieval/MR_valid_imgs.tsv  
  inflating: ./data/Multimodal_Retrieval/MR_test_imgs.tsv  


In [4]:
!unzip -d ./weights ./weights/chinese_roberta_wwm_large_ext_pytorch.zip

Archive:  ./weights/chinese_roberta_wwm_large_ext_pytorch.zip
  inflating: ./weights/pytorch_model.bin  
  inflating: ./weights/vocab.txt     
  inflating: ./weights/bert_config.json  


## 准备 CLIP 和 BERT 权重

In [5]:
!python3 src/preprocess/transform_openai_pretrain_weights.py --raw-ckpt-path ./weights/ViT-B-16.pt --new-ckpt-path ./weights/ViT-B-16.state_dict.pt

Params:
  new_ckpt_path: ./weights/ViT-B-16.state_dict.pt
  raw_ckpt_path: ./weights/ViT-B-16.pt
Transformed openai ckpt ./weights/ViT-B-16.pt to ./weights/ViT-B-16.state_dict.pt!
Done!


## 准备图片数据集

In [8]:
!python3 src/preprocess/transform_images.py --data_dir ./data/Multimodal_Retrieval --image_resolution 224

begin to transform train split
12000it [00:13, 888.18it/s]
finished transforming 12000 images for train split, the output is saved at ./data/Multimodal_Retrieval/MR_train_imgs.224.npz
begin to transform valid split
12000it [00:15, 798.21it/s]
finished transforming 12000 images for valid split, the output is saved at ./data/Multimodal_Retrieval/MR_valid_imgs.224.npz
begin to transform test split
12000it [00:16, 713.72it/s]
finished transforming 12000 images for test split, the output is saved at ./data/Multimodal_Retrieval/MR_test_imgs.224.npz
done!


In [10]:
filename='./data/Multimodal_Retrieval/MR_train_imgs.tsv'   #文件路径
total = len(open(filename).readlines())
print('The total train imgs is ',total)

filename='./data/Multimodal_Retrieval/MR_valid_imgs.tsv'   #文件路径
total = len(open(filename).readlines())
print('The total val imgs is ',total)

filename='./data/Multimodal_Retrieval/MR_test_imgs.tsv'   #文件路径
total = len(open(filename).readlines())
print('The total test imgs is ',total)

The total train imgs is  129380
The total val imgs is  29806
The total test imgs is  30399


## 由于我们使用的是不完全数据集，所以要把标注文件和不完全数据集对齐

In [12]:
import json
import numpy as np
from tqdm import tqdm

img_filename = "./data/Multimodal_Retrieval/MR_train_imgs.224.npz"
jsonl_filename = "./data/Multimodal_Retrieval/MR_train_queries.jsonl"
tiny_jsonl_filename = "./data/Multimodal_Retrieval/MR_train_queries_tiny.jsonl"

imgs = np.load(img_filename, "r")

all_new_json = []

with open(jsonl_filename, "r") as fin:
    for line in tqdm(fin):
        new_json_line = {}
        obj = json.loads(line.strip())
        new_json_line['query_id'] = obj['query_id']
        new_json_line['query_text'] = obj['query_text']
        new_json_line['item_ids'] = []
        for target in obj['item_ids']:
            #print(imgs[str(img_id)])
            if(str(target) in imgs):
                new_json_line['item_ids'].append(target)
        if(len(new_json_line['item_ids'])>0):
            all_new_json.append(new_json_line)
    
print('小数据集的query-相关商品对数量：',len(all_new_json))

with open(tiny_jsonl_filename,'w') as f:
    for i in range(len(all_new_json)):
        f.write(json.dumps(all_new_json[i], ensure_ascii=False))
        f.write('\n')

248786it [00:57, 4301.49it/s]


小数据集的query-相关商品对数量： 23481


In [11]:
# 验证集
img_filename = "./data/Multimodal_Retrieval/MR_valid_imgs.224.npz"
jsonl_filename = "./data/Multimodal_Retrieval/MR_valid_queries.jsonl"
tiny_jsonl_filename = "./data/Multimodal_Retrieval/MR_valid_queries_tiny.jsonl"

imgs = np.load(img_filename, "r")

all_new_json = []

with open(jsonl_filename, "r") as fin:
    for line in tqdm(fin):
        new_json_line = {}
        obj = json.loads(line.strip())
        new_json_line['query_id'] = obj['query_id']
        new_json_line['query_text'] = obj['query_text']
        new_json_line['item_ids'] = []
        for target in obj['item_ids']:
            #print(imgs[str(img_id)])
            if(str(target) in imgs):
                new_json_line['item_ids'].append(target)
        if(len(new_json_line['item_ids'])>0):
            all_new_json.append(new_json_line)
    
print('小数据集的query-相关商品对数量：',len(all_new_json))

with open(tiny_jsonl_filename,'w') as f:
    for i in range(len(all_new_json)):
        #f.write(str(all_new_json[i]))
        f.write(json.dumps(all_new_json[i], ensure_ascii=False))
        f.write('\n')

5008it [00:12, 416.81it/s]

小数据集的query-相关商品对数量： 4750


## 训练

In [ ]:
!python3 -u src/training/main.py \
    --save-frequency 5 \
    --train-data="./data/Multimodal_Retrieval/MR_train_queries_tiny.jsonl"  \
    --train-img="./data/Multimodal_Retrieval/MR_train_imgs.224.npz"  \
    --val-data="./data/Multimodal_Retrieval/MR_valid_queries_tiny.jsonl"  \
    --val-img="./data/Multimodal_Retrieval/MR_valid_imgs.224.npz"  \
    --clip-weight-path="./weights/ViT-B-16.state_dict.pt" \
    --bert-weight-path="./weights/pytorch_model.bin" \
    --warmup 500 \
    --batch-size=32 \
    --lr=8e-5 \
    --wd=0.001 \
    --epochs=10 \
    --model ViT-B-16 \
    --gpu 0

Loading model from src/training/model_configs/ViT-B-16.json
2022-11-08,05:23:55 | INFO | Rank 0 | Params:
2022-11-08,05:23:55 | INFO | Rank 0 |   aggregate: True
2022-11-08,05:23:55 | INFO | Rank 0 |   batch_size: 32
2022-11-08,05:23:55 | INFO | Rank 0 |   bert_weight_path: ./weights/pytorch_model.bin
2022-11-08,05:23:55 | INFO | Rank 0 |   beta1: 0.9
2022-11-08,05:23:55 | INFO | Rank 0 |   beta2: 0.98
2022-11-08,05:23:55 | INFO | Rank 0 |   checkpoint_path: ./logs/lr=8e-05_wd=0.001_agg=True_model=ViT-B-16_batchsize=32_date=2022-11-08-05-23-55/checkpoints
2022-11-08,05:23:55 | INFO | Rank 0 |   clip_weight_path: ./weights/ViT-B-16.state_dict.pt
2022-11-08,05:23:55 | INFO | Rank 0 |   copy_codebase: False
2022-11-08,05:23:55 | INFO | Rank 0 |   debug: False
2022-11-08,05:23:55 | INFO | Rank 0 |   dist_backend: nccl
2022-11-08,05:23:55 | INFO | Rank 0 |   dist_url: tcp://127.0.0.1:6100
2022-11-08,05:23:55 | INFO | Rank 0 |   distributed: False
2022-11-08,05:23:55 | INFO | Rank 0 |   dp: 